-----------------------------
## Nama : Rafi Fadhlillah
## NIM : 121450143
-----------------------------

In [1]:
import json
import numpy as np
import pandas as pd
import time

# Fungsi aktivasi
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Fungsi update SGD
def sgd_update(weight, gradient, learning_rate):
    return weight + learning_rate * gradient

# Fungsi update Momentum
def momentum_update(weight, gradient, learning_rate, velocity, momentum):
    velocity = momentum * velocity + learning_rate * gradient
    return weight + velocity, velocity

# Fungsi update RMSProp
def rmsprop_update(weight, gradient, v, learning_rate, beta2, epsilon=1e-8):
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    weight -= learning_rate * gradient / (np.sqrt(v) + epsilon)
    return weight, v

# Fungsi update Adam
def adam_update(weight, gradient, m, v, t, learning_rate, beta1, beta2, epsilon=1e-8):
    m = beta1 * m + (1 - beta1) * gradient
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    weight += learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    return weight, m, v



# Neural network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, optimizer='sgd'):
        # Inisialisasi bobot dan bias
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

        # Setup variabel terkait optimizer
        self.optimizer = optimizer
        self.velocity_ih = np.zeros_like(self.weights_input_hidden)
        self.velocity_ho = np.zeros_like(self.weights_hidden_output)
        self.m_ih = np.zeros_like(self.weights_input_hidden)
        self.v_ih = np.zeros_like(self.weights_input_hidden)
        self.m_ho = np.zeros_like(self.weights_hidden_output)
        self.v_ho = np.zeros_like(self.weights_hidden_output)
        self.beta1, self.beta2 = 0.9, 0.999
        self.t = 1

    def forward(self, inputs):
        # Propagasi maju untuk batch
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = sigmoid(self.output_input)
        return self.predicted_output

    def backward(self, inputs, targets, learning_rate):
        # Backpropagation untuk batch
        error = targets - self.predicted_output
        delta_output = error * sigmoid_derivative(self.predicted_output)
        error_hidden = delta_output.dot(self.weights_hidden_output.T)
        delta_hidden = error_hidden * sigmoid_derivative(self.hidden_output)

        # Update bobot menggunakan optimizer yang dipilih
        if self.optimizer == 'sgd':
            self.weights_hidden_output = sgd_update(self.weights_hidden_output,
                                                     np.dot(self.hidden_output.T, delta_output), learning_rate)
            self.weights_input_hidden = sgd_update(self.weights_input_hidden,
                                                    np.dot(inputs.T, delta_hidden), learning_rate)

        elif self.optimizer == 'momentum':
            self.weights_hidden_output, self.velocity_ho = momentum_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                learning_rate, self.velocity_ho, momentum=0.9
            )
            self.weights_input_hidden, self.velocity_ih = momentum_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                learning_rate, self.velocity_ih, momentum=0.9
            )

        elif self.optimizer == 'adam':
            self.weights_hidden_output, self.m_ho, self.v_ho = adam_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                self.m_ho, self.v_ho, self.t, learning_rate, self.beta1, self.beta2
            )
            self.weights_input_hidden, self.m_ih, self.v_ih = adam_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                self.m_ih, self.v_ih, self.t, learning_rate, self.beta1, self.beta2
            )
            self.t += 1

        elif self.optimizer == 'rmsprop':
            self.weights_hidden_output, self.v_ho = rmsprop_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                self.v_ho, learning_rate, self.beta2
            )
            self.weights_input_hidden, self.v_ih = rmsprop_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                self.v_ih, learning_rate, self.beta2
            )

    def train(self, training_data, targets, epochs, learning_rate, batch_size):
        error_history = []  # Untuk menyimpan error di setiap epoch
        for epoch in range(epochs):
            total_error = 0
            for i in range(0, len(training_data), batch_size):
                # Buat batch
                batch_inputs = training_data[i:i + batch_size]
                batch_targets = targets[i:i + batch_size].reshape(-1, 1)

                # Forward pass
                self.forward(batch_inputs)

                # Backward pass
                self.backward(batch_inputs, batch_targets, learning_rate)

                # Hitung error untuk batch
                total_error += np.mean(np.square(batch_targets - self.predicted_output))

            error_history.append(total_error / (len(training_data) / batch_size))

            # Print error setiap 10 epoch
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Error: {total_error / (len(training_data) / batch_size)}")
        # return error_history

    def predict(self, inputs):
        return self.forward(inputs)

# Menggunakan dataset baru untul soal latihan
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/transfusion_data.csv'
data = pd.read_csv(url)

#data = pd.read_csv("heart_diseases.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# # Test network
# for inputs in training_data:
#     prediction = nn.predict(inputs.reshape(1, -1))  # Ubah input ke bentuk batch
#     print(f"Predicted Output: {prediction}")




# # Fungsi untuk melatih dengan optimizer tertentu
# def train_with_optimizer(optimizer, input_size, hidden_size, output_size,
#                          training_data, targets, epochs, learning_rate, batch_size):
#     # Pilih optimizer langsung di sini
#     nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
#     start_time = time.time()
#     error_history = nn.train(training_data, targets, epochs, learning_rate, batch_size)
#     training_time = time.time() - start_time
#     return error_history, training_time

# for inputs in training_data:
#     prediction = nn.predict(inputs)
#     print(f"Input: {inputs}, Predicted Output: {prediction}")


# # Latih model dengan berbagai optimizer
# optimizers = ['sgd', 'momentum', 'adam', 'rmsprop']
# results = {}

# for opt in optimizers:
#     print(f"Training with {opt} optimizer...")
#     error_history, training_time = train_with_optimizer(
#         opt, input_size, hidden_size, output_size,
#         training_data, targets, epochs, learning_rate, batch_size
#     )
#     results[opt] = {
#         'error_history': error_history,
#         'training_time': training_time
#     }

# # Simpan hasil ke file JSON
# with open('optimizer_results_single_line.json', 'w') as f:
#     for optimizer, data in results.items():
#         json_line = json.dumps({optimizer: data}, separators=(',', ':'))
#         f.write(json_line + '\n')

# print("Results saved to optimizer_results_single_line.json")

# Nomor 1
kita dapat mengubah parameter jumlah neuron pada hidden layer, jumlah output learning rate, epochs dan batch size sesuai yang kita inginkan menggunakan code dibawah

In [2]:
#NOMOR 1
# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 5  # Jumlah node di hidden layer NOMOR 1a
output_size = 2  # Jumlah output
learning_rate = 0.01
epochs = 200 #NOMOR 1b
batch_size = 13 # NOMOR 1c

# Nomor 2
berikut adalah dataset baru untuk latihan, kita dapat menyesuaikan parameter batch size dan epochs yang berbeda menggunakan code nomor 1

In [3]:
# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 5
output_size = 2
learning_rate = 0.01
epochs = 200 #ubah disini
batch_size = 8 #ubah disini

In [4]:
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/transfusion_data.csv'
data = pd.read_csv(url)

#data = pd.read_csv("transfusion_data.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

In [5]:
nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='momentum')
nn.train(training_data, targets, epochs, learning_rate, batch_size)

<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.18043652803419152
Epoch 10, Error: 0.1891307844026584
Epoch 20, Error: 0.1890710732682616
Epoch 30, Error: 0.18903305517199284
Epoch 40, Error: 0.18900082846469182
Epoch 50, Error: 0.1889733184960266
Epoch 60, Error: 0.18894966504974353
Epoch 70, Error: 0.1889291822633684
Epoch 80, Error: 0.18891132303122676
Epoch 90, Error: 0.18889564952765306
Epoch 100, Error: 0.1888818096643456
Epoch 110, Error: 0.1888695186390624
Epoch 120, Error: 0.18885854460601953
Epoch 130, Error: 0.18884869758351047
Epoch 140, Error: 0.18883982086409667
Epoch 150, Error: 0.18883178434482986
Epoch 160, Error: 0.18882447932682628
Epoch 170, Error: 0.1888178144399853
Epoch 180, Error: 0.18881171243156558
Epoch 190, Error: 0.18880610762062194


In [6]:
epochs2 = 375 #coba kita naikkan epochs
batch_size2 = 12 #coba kita naikkan batch size
learning_rate2 = 0.02

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='momentum')
nn.train(training_data, targets, epochs2, learning_rate, batch_size2)

<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.42116870296986003
Epoch 10, Error: 0.1854987911062511
Epoch 20, Error: 0.1854987911062498
Epoch 30, Error: 0.1854987911062498
Epoch 40, Error: 0.1854987911062498
Epoch 50, Error: 0.1854987911062498
Epoch 60, Error: 0.1854987911062498
Epoch 70, Error: 0.1854987911062498
Epoch 80, Error: 0.1854987911062498
Epoch 90, Error: 0.1854987911062498
Epoch 100, Error: 0.1854987911062498
Epoch 110, Error: 0.1854987911062498
Epoch 120, Error: 0.1854987911062498
Epoch 130, Error: 0.1854987911062498
Epoch 140, Error: 0.1854987911062498
Epoch 150, Error: 0.1854987911062498
Epoch 160, Error: 0.1854987911062498
Epoch 170, Error: 0.1854987911062498
Epoch 180, Error: 0.1854987911062498
Epoch 190, Error: 0.1854987911062498
Epoch 200, Error: 0.1854987911062498
Epoch 210, Error: 0.1854987911062498
Epoch 220, Error: 0.1854987911062498
Epoch 230, Error: 0.1854987911062498
Epoch 240, Error: 0.1854987911062498
Epoch 250, Error: 0.1854987911062498
Epoch 260, Error: 0.1854987911062498
Epoch 270, 

In [7]:
epochs3 = 100 #coba kita turunkan epochs
batch_size3 = 6 #coba kita turunkan batch size
learning_rate3 = 0.03

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='momentum')
nn.train(training_data, targets, epochs3, learning_rate, batch_size3)

Epoch 0, Error: 0.2152307840568079


<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 10, Error: 0.1871389700051244
Epoch 20, Error: 0.18713897000512425
Epoch 30, Error: 0.18713897000512403
Epoch 40, Error: 0.18713897000512386
Epoch 50, Error: 0.18713897000512364
Epoch 60, Error: 0.18713897000512353
Epoch 70, Error: 0.18713897000512333
Epoch 80, Error: 0.18713897000512322
Epoch 90, Error: 0.1871389700051231


## Menggunakan optimizer momentum diperoleh perbandingannya sebagai berikut :
#### Epochs = 200, Batchsize = 8
- Epoch 0, Error: 0.207926857517832
- Epoch 190, Error: 0.18957127294658063

#### Epochs = 375, Batchsize = 12
- Epoch 0, Error: 0.19819548958848104
- Epoch 370, Error: 0.18835926000506953

#### Epochs = 100, Batchsize = 6
- Epoch 0, Error: 0.17810895111720745
- Epoch 90, Error: 0.18713897003550062

# Nomor 3

### Menggunakan adam

In [8]:
nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
nn.train(training_data, targets, epochs, learning_rate, batch_size)

<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.23327849759021413
Epoch 10, Error: 0.18383431793407595
Epoch 20, Error: 0.1839018396373308
Epoch 30, Error: 0.1839226766399301
Epoch 40, Error: 0.18393022488238803
Epoch 50, Error: 0.1839331130897959
Epoch 60, Error: 0.18393424078286336
Epoch 70, Error: 0.18393468449604927
Epoch 80, Error: 0.1839348596052519
Epoch 90, Error: 0.18393492878761103
Epoch 100, Error: 0.1839349561241168
Epoch 110, Error: 0.18393496691502945
Epoch 120, Error: 0.18393497115936144
Epoch 130, Error: 0.18393497281218904
Epoch 140, Error: 0.18393497344047843
Epoch 150, Error: 0.18393497366681605
Epoch 160, Error: 0.1839349737390286
Epoch 170, Error: 0.18393497375519247
Epoch 180, Error: 0.18393497375321907
Epoch 190, Error: 0.18393497374699372


### Menggunakan sgd

In [9]:
nn2 = NeuralNetwork(input_size, hidden_size, output_size, optimizer='sgd')
nn2.train(training_data, targets, epochs, learning_rate, batch_size)

<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.28094827491924196
Epoch 10, Error: 0.18184168372942347
Epoch 20, Error: 0.1818523181833653
Epoch 30, Error: 0.18184421251857932
Epoch 40, Error: 0.18183641679813578
Epoch 50, Error: 0.18182904549741297
Epoch 60, Error: 0.1818220717535361
Epoch 70, Error: 0.18181546954903102
Epoch 80, Error: 0.18180921464853972
Epoch 90, Error: 0.1818032844979258
Epoch 100, Error: 0.181797658115088
Epoch 110, Error: 0.18179231598090015
Epoch 120, Error: 0.18178723993012966
Epoch 130, Error: 0.18178241304050458
Epoch 140, Error: 0.18177781951510497
Epoch 150, Error: 0.1817734445468191
Epoch 160, Error: 0.18176927413771027
Epoch 170, Error: 0.1817652948014262
Epoch 180, Error: 0.18176149292923185
Epoch 190, Error: 0.18175785299635105


## Perbandingan Waktu running

In [10]:
def train_and_time(optimizer, input_size, hidden_size, output_size,
                         training_data, targets, epochs, learning_rate, batch_size):
    nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer=optimizer)
    start_time = time.time()
    nn.train(training_data, targets, epochs, learning_rate, batch_size)
    training_time = time.time() - start_time
    return training_time


optimizers = ['sgd', 'adam']
results = {}

for opt in optimizers:
    print(f"Training with {opt} optimizer...")
    training_time = train_and_time(
        opt, input_size, hidden_size, output_size,
        training_data, targets, epochs, learning_rate, batch_size
    )
    results[opt] = training_time

print("\nTraining Time Comparisons:")
for optimizer, time_taken in results.items():
    print(f"{optimizer}: {time_taken:.4f} seconds")

Training with sgd optimizer...
Epoch 0, Error: 0.1874516049839636
Epoch 10, Error: 0.18192168076353
Epoch 20, Error: 0.18192176506792912


<ipython-input-1-509ef1468f71>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 30, Error: 0.18192176512121955
Epoch 40, Error: 0.1819217651212528
Epoch 50, Error: 0.18192176512125285
Epoch 60, Error: 0.18192176512125285
Epoch 70, Error: 0.18192176512125285
Epoch 80, Error: 0.18192176512125285
Epoch 90, Error: 0.18192176512125285
Epoch 100, Error: 0.18192176512125285
Epoch 110, Error: 0.18192176512125285
Epoch 120, Error: 0.1819217651212528
Epoch 130, Error: 0.1819217651212528
Epoch 140, Error: 0.18192176512125285
Epoch 150, Error: 0.18192176512125285
Epoch 160, Error: 0.18192176512125285
Epoch 170, Error: 0.18192176512125285
Epoch 180, Error: 0.18192176512125285
Epoch 190, Error: 0.18192176512125285
Training with adam optimizer...
Epoch 0, Error: 0.2619940589443572
Epoch 10, Error: 0.18368794969261007
Epoch 20, Error: 0.18385457537010153
Epoch 30, Error: 0.18390509611541456
Epoch 40, Error: 0.18392339635857935
Epoch 50, Error: 0.1839304192227073
Epoch 60, Error: 0.18393317169910464
Epoch 70, Error: 0.18393425915107686
Epoch 80, Error: 0.1839346901178205
Epo

dari segi waktu dapat dilihat bahwa komputasi dari adam lebih lama dibandingkan dengan sgd